# Requirements

- Python3.5-3.8
- Tesnorflow (`pip install tensorflow`)
- Tensorflow Dataset (`pip install tensorflow_datasets`)
- Requsets (`pip install requests`)
- Easytensor (`pip install easytensor`)

In [ ]:
! pip install tensorflow tensorflow_datasets requests easytensor

# Train a simple MNIST model
Here, we will train a simple model that can classify english digits from 1 to 9. The MNIST dataset is made up of small images that are 28x28 pixels.

The full explanation of how this model is trained can be found [here](https://www.tensorflow.org/datasets/keras_example).

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

In [ ]:
import os
export_path = os.path.join(os.getcwd(), "my_model")
print("export_path: {}".format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

# Upload to EasyTensor
Now that we have the model exported, we can upload it to cloud and have EasyTensor serve it for us!

In [ ]:
import easytensor
model_id, access_token = easytensor.upload_model("MNIST Model", export_path)

That's it! The model is now in the cloud with one line of code. Each model will have an ID and an access token. Anyone with the access token can send requests to the model, so make sure you store it somewhere safe!

In [ ]:
print("model ID:", model_id)
print("access token:", access_token)

# Use the model
Now let's try to use the model via a simple http request. We will use a random image from the training set.

In [ ]:
image_batches = list(map(lambda x: x[0], ds_train)) # shape (x, 128, 28, 28, 1)
label_batches = list(map(lambda x: x[1], ds_train)) # shape (y, 128, 28, 28, 1)
print("number of image batches:", len(image_batches))
print("Each batch contains", image_batches[0].shape[0], "images")
print("Each image has shape", image_batches[0][0].shape)

In [ ]:
response = requests.post(
    "https://app.easytensor.com/query/",
    json={
        "instances": [
            # use the first picture of the first batch
            # make it a serializable list
            images[0][0].numpy().tolist()
        ]
    },
    headers={"accessToken": access_token}
)
print(response.text)